## Convolutional Neural Networks

Again, I am not going to even try to do a better job than this post...:

https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/

## Python Conv Net

In [52]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import numpy as np

In [38]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

def scale_image(images):
    images = images.reshape(images.shape[0], 28, 28, 1)
    images = images.astype('float32')
    scaled_images = images / 255
    return scaled_images

x_train = scale_image(x_train)
x_test = scale_image(x_test)

In [39]:
x_train.shape

(60000, 28, 28, 1)

In [46]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_20 (Dense)             (None, 128)               32896     
__________

In [48]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy')

In [49]:
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 33s - loss: 0.1379 - val_loss: 0.0535
Epoch 2/5
48000/48000 [==============================] - 35s - loss: 0.0448 - val_loss: 0.0503
Epoch 3/5
48000/48000 [==============================] - 35s - loss: 0.0325 - val_loss: 0.0416
Epoch 4/5
48000/48000 [==============================] - 35s - loss: 0.0243 - val_loss: 0.0512s: 0.
Epoch 5/5
48000/48000 [==============================] - 35s - loss: 0.0207 - val_loss: 0.0348


In [53]:
test_predictions = np.argmax(model.predict(x_test),1)

In [54]:
confusion_matrix(y_test, test_predictions)

array([[ 978,    0,    0,    0,    0,    0,    0,    1,    1,    0],
       [   0, 1132,    0,    1,    0,    0,    0,    2,    0,    0],
       [   0,    0, 1028,    0,    0,    0,    0,    3,    1,    0],
       [   0,    0,    2, 1001,    0,    4,    0,    1,    2,    0],
       [   0,    0,    1,    0,  979,    0,    0,    0,    0,    2],
       [   2,    0,    0,    5,    0,  883,    1,    1,    0,    0],
       [   6,    3,    3,    0,    2,    1,  942,    0,    1,    0],
       [   0,    2,    2,    0,    0,    0,    0, 1023,    0,    1],
       [   1,    0,    2,    0,    0,    1,    0,    1,  966,    3],
       [   0,    0,    1,    0,    8,    4,    0,    2,    3,  991]])

In [55]:
np.sum(y_test == test_predictions) / test_predictions.shape

array([ 0.9923])